# STS Benchmark Datasets

### Preparation

Setup all required libraries

In [1]:
import logging
import re
import sys

import numpy as np
import pandas as pd

from fse import Average, SIF, uSIF, Vectors, FTVectors, CSplitIndexedList

from re import sub

from scipy.stats import pearsonr

from nltk import word_tokenize

logging.basicConfig(format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s', level=logging.INFO)

Next, we require the sentences from the STS benchmark dataset.

In [2]:
file= "../evaluation/sts-test.csv"
similarities, sent_a, sent_b = [], [], []
with open(file, "r") as f:
    for l in f:
        line = l.rstrip().split("\t")
        similarities.append(float(line[4]))
        sent_a.append(line[5])
        sent_b.append(line[6])
similarities = np.array(similarities)
assert len(similarities) == len(sent_a) == len(sent_b)
task_length = len(similarities)

for i, obj in enumerate(zip(similarities, sent_a, sent_b)):
    print(f"{i}\tSim: {obj[0].round(3):.1f}\t{obj[1]:40s}\t{obj[2]:40s}\t")
    if i == 4:
        break

0	Sim: 2.5	A girl is styling her hair.             	A girl is brushing her hair.            	
1	Sim: 3.6	A group of men play soccer on the beach.	A group of boys are playing soccer on the beach.	
2	Sim: 5.0	One woman is measuring another woman's ankle.	A woman measures another woman's ankle. 	
3	Sim: 4.2	A man is cutting up a cucumber.         	A man is slicing a cucumber.            	
4	Sim: 1.5	A man is playing a harp.                	A man is playing a keyboard.            	


Each of these sentence requires some preparation (i.e. tokenization) to be used in the core input formats.
To reproduce the results from the uSIF paper this part is taken from https://github.com/kawine/usif/blob/master/usif.py

In [3]:
not_punc = re.compile('.*[A-Za-z0-9].*')

def prep_token(token):
    t = token.lower().strip("';.:()").strip('"')
    t = 'not' if t == "n't" else t
    return re.split(r'[-]', t)

def prep_sentence(sentence):
    tokens = []
    for token in word_tokenize(sentence):
        if not_punc.match(token):
            tokens = tokens + prep_token(token)
    return tokens

Next we define the IndexedList object. The IndexedList extends the previously constructed sent_a and sent_b list together. We additionally provide a custom function "prep_sentence" which performs all the prepocessing for a single sentence. Therefore we need the extention **CSplitIndexedList**, which provides you the option to provide a custom split function

In [4]:
sentences = CSplitIndexedList(sent_a, sent_b, custom_split=prep_sentence)

The IndexedList returns the core object required for fse to train a sentence embedding: A tuple. This object constists of words (a list of strings) and its corresponding index. The latter is important if multiple cores access the input queue simultaneously. Thus it must be always provided. The index represents the row in the matrix where it can later be found.

In [5]:
sentences[0]

(['a', 'girl', 'is', 'styling', 'her', 'hair'], 0)

Note, that IndexedList does not convert the sentences inplace but only on calling the __getitem__ method in order to turn the sentence into a tuple. You can access the original sentence using

In [6]:
sentences.items[0]

'A girl is styling her hair.'

### Loading the models

It is required for us to load the models as BaseKeyedVectors or as an BaseWordEmbeddingsModel. For this notebook, I already converted the models to a BaseKeyedVectors instance and saved the corresponding instance on my external harddrive. You have to replicate these steps yourself, because getting all the files can be a bit difficult, as the total filesize is around 15 GB.

The following code performs a disk-to-ram training. Passing a path to __wv_mapfile_path__ will store the corresponding word vectors (wv) as a numpy memmap array. This is required, because loading all vectors into ram would would take up a lot of storage unecessary. The wv.vectors file will be replace by its memmap representation, which is why the next models do not require the wv_mapfile_path argument, as they access the same memmap object.

The lang_freq="en" induces the frequencies of the words according to the wordfreq package. This functionality allows you to work with pre-trained embeddings which don't come with frequency information. The method overwrites the counts in the glove.wv.vocab class, so that all further models also benefit from this induction.

In [7]:
models = {}
model_names = [
    "fasttext-wiki-news-subwords-300",
    "glove-twitter-100",
    "glove-twitter-200",
    "glove-twitter-25",
    "glove-twitter-50",
    "glove-wiki-gigaword-100",
    "glove-wiki-gigaword-200",
    "glove-wiki-gigaword-300",
    "glove-wiki-gigaword-50",
    "paragram-25",
    "paragram-300-sl999",
    "paragram-300-ws353",
    "paranmt-300",
    "word2vec-google-news-300",
]

vectors = {}

for name in model_names:
    vectors[name] = Vectors.from_pretrained(name, mmap="r")
    
ft = "fasttext-crawl-subwords-300"
vectors[ft] = FTVectors.from_pretrained(ft, mmap="r")

2021-12-03 09:19:01,771 : MainThread : INFO : loading Vectors object from /home/oborchers/.cache/huggingface/hub/fse__fasttext-wiki-news-subwords-300.ef21870476cd93435d83140fcf6e7171b517e337/fasttext-wiki-news-subwords-300.model
2021-12-03 09:19:03,869 : MainThread : INFO : loading vectors from /home/oborchers/.cache/huggingface/hub/fse__fasttext-wiki-news-subwords-300.ef21870476cd93435d83140fcf6e7171b517e337/fasttext-wiki-news-subwords-300.model.vectors.npy with mmap=r
2021-12-03 09:19:03,872 : MainThread : INFO : setting ignored attribute vectors_norm to None
2021-12-03 09:19:03,873 : MainThread : INFO : loaded /home/oborchers/.cache/huggingface/hub/fse__fasttext-wiki-news-subwords-300.ef21870476cd93435d83140fcf6e7171b517e337/fasttext-wiki-news-subwords-300.model
2021-12-03 09:19:06,810 : MainThread : INFO : loading Vectors object from /home/oborchers/.cache/huggingface/hub/fse__glove-twitter-100.7ef1c21d9bf90598a0c618c041d9817e50250183/glove-twitter-100.model
2021-12-03 09:19:09,430

In [26]:
algos = {
    "CBOW" : (Average, {}),
    "SIF-10" : (SIF, {"components" : 10}),
    "uSIF": (uSIF, {"length": 11}),
}
import itertools
combinations = list(itertools.product(algos.keys(), vectors.keys()))

## Computation of the results for the STS benchmark

We are finally able to compute the STS benchmark values for all models.

In [27]:
# This function is used to compute the similarities between two sentences.
# Task length is the length of the sts dataset.
def compute_similarities(task_length, model):
    sims = []
    for i, j in zip(range(task_length), range(task_length, 2*task_length)):
        sims.append(model.sv.similarity(i,j))
    return sims

In [28]:
from collections import defaultdict
results = defaultdict(list)

for algo, name in combinations:
    class_obj = algos[algo][0]
    args = algos[algo][1]
    vec = vectors[name]
    m = class_obj(vec, lang_freq="en", **args)
    m.train(sentences)
    r = pearsonr(similarities, compute_similarities(task_length, m))[0].round(4) * 100
    
    results["algo"].append(algo)
    results["vecs"].append(name)
    
    if len(args) > 0:
        name = list(args.keys())[0]
        value = args[name]
        results["params"].append(f"{name}={value}")
    else:
        results["params"].append("")
    
    results["score"].append(r)
    print(algo, name, f"{r:2.2f}")

2021-12-03 09:38:51,576 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en
2021-12-03 09:38:52,415 : MainThread : INFO : scanning all indexed sentences and their word counts
2021-12-03 09:38:52,773 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2021-12-03 09:38:53,082 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 999999 vocabulary: 1151 MB (1 GB)
2021-12-03 09:38:53,084 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2021-12-03 09:38:53,130 : MainThread : INFO : begin training
2021-12-03 09:38:53,488 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2021-12-03 09:38:53,489 : MainThread : INFO : training on 2758 effective sentences with 27172 effective words took 0s with 7686 sentences/s
2021-12-03 09:38:53,532 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency

CBOW fasttext-wiki-news-subwords-300 26.08


2021-12-03 09:38:54,580 : MainThread : INFO : scanning all indexed sentences and their word counts
2021-12-03 09:38:54,910 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2021-12-03 09:38:55,347 : MainThread : INFO : estimated memory for 2758 sentences with 100 dimensions and 1193514 vocabulary: 460 MB (0 GB)
2021-12-03 09:38:55,348 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2021-12-03 09:38:55,381 : MainThread : INFO : begin training
2021-12-03 09:38:55,791 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2021-12-03 09:38:55,792 : MainThread : INFO : training on 2758 effective sentences with 26828 effective words took 0s with 6698 sentences/s
2021-12-03 09:38:55,822 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en


CBOW glove-twitter-100 33.81


2021-12-03 09:38:56,793 : MainThread : INFO : scanning all indexed sentences and their word counts
2021-12-03 09:38:57,153 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2021-12-03 09:38:57,578 : MainThread : INFO : estimated memory for 2758 sentences with 200 dimensions and 1193514 vocabulary: 917 MB (0 GB)
2021-12-03 09:38:57,580 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2021-12-03 09:38:57,629 : MainThread : INFO : begin training
2021-12-03 09:38:57,973 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2021-12-03 09:38:57,974 : MainThread : INFO : training on 2758 effective sentences with 26828 effective words took 0s with 8008 sentences/s
2021-12-03 09:38:58,003 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en


...

uSIF length 79.02


2021-12-03 09:41:01,070 : MainThread : INFO : scanning all indexed sentences and their word counts
2021-12-03 09:41:01,448 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2021-12-03 09:41:02,515 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 3000000 vocabulary: 3447 MB (3 GB)
2021-12-03 09:41:02,516 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2021-12-03 09:41:02,529 : MainThread : INFO : pre-computing uSIF weights for 3000000 words
2021-12-03 09:41:09,927 : MainThread : INFO : begin training
2021-12-03 09:41:10,374 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2021-12-03 09:41:10,733 : MainThread : INFO : computing 5 principal components took 0s
2021-12-03 09:41:10,740 : MainThread : INFO : removing 5 principal components took 0s
2021-12-03 09:41:10,742 : MainThread : INFO : training on 2758 effective sentences with 23116 effective wor

uSIF length 67.15


2021-12-03 09:41:12,490 : MainThread : INFO : scanning all indexed sentences and their word counts
2021-12-03 09:41:12,816 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2021-12-03 09:41:13,574 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 2000000 vocabulary: 6877 MB (6 GB)
2021-12-03 09:41:13,575 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2021-12-03 09:41:13,591 : MainThread : INFO : pre-computing uSIF weights for 2000000 words
2021-12-03 09:41:18,512 : MainThread : INFO : begin training
2021-12-03 09:41:18,980 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2021-12-03 09:41:19,312 : MainThread : INFO : computing 5 principal components took 0s
2021-12-03 09:41:19,317 : MainThread : INFO : removing 5 principal components took 0s
2021-12-03 09:41:19,318 : MainThread : INFO : training on 2758 effective sentences with 27528 effective wor

uSIF length 69.55


In [29]:
df = pd.DataFrame.from_dict(results, orient="columns").sort_values("score", ascending=False)
df.head()

,algo,vecs,params,score
12,CBOW,paranmt-300,,79.82
42,uSIF,paranmt-300,length=11,79.02
27,SIF-10,paranmt-300,components=10,76.76
25,SIF-10,paragram-300-sl999,components=10,74.27
26,SIF-10,paragram-300-ws353,components=10,74.08


If you closely study the values above you will find that:
- SIF-Glove is almost equivalent to the values reported in http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark
- CBOW-Paranmt is a little better than ParaNMT Word Avg. in https://www.aclweb.org/anthology/W18-3012
- uSIF-Paranmt is a little worse than ParaNMT+UP in https://www.aclweb.org/anthology/W18-3012
- uSIF-Paragram is a little worse than PSL+UP in https://www.aclweb.org/anthology/W18-3012

However, I guess those differences might arise due to differences in preprocessing. Too bad we didn't hit 80. If you have any ideas why those values don't match exactly, feel free to contact me anytime.

In [36]:
for row in df.iterrows():
    row = row[1]
    print(
        f"`{row.algo}` | `{row.vecs}` | {row.params} | {row.score:2.2f}"
    )    

`CBOW` | `paranmt-300` |  | 79.82
`uSIF` | `paranmt-300` | length=11 | 79.02
`SIF-10` | `paranmt-300` | components=10 | 76.76
`SIF-10` | `paragram-300-sl999` | components=10 | 74.27
`SIF-10` | `paragram-300-ws353` | components=10 | 74.08
`SIF-10` | `fasttext-crawl-subwords-300` | components=10 | 73.54
`uSIF` | `paragram-300-sl999` | length=11 | 73.09
`SIF-10` | `fasttext-wiki-news-subwords-300` | components=10 | 72.24
`uSIF` | `paragram-300-ws353` | length=11 | 71.90
`SIF-10` | `glove-twitter-200` | components=10 | 71.67
`SIF-10` | `glove-wiki-gigaword-300` | components=10 | 71.43
`SIF-10` | `word2vec-google-news-300` | components=10 | 71.17
`SIF-10` | `glove-wiki-gigaword-200` | components=10 | 70.73
`SIF-10` | `glove-twitter-100` | components=10 | 69.70
`uSIF` | `fasttext-crawl-subwords-300` | length=11 | 69.55
`uSIF` | `fasttext-wiki-news-subwords-300` | length=11 | 69.05
`SIF-10` | `glove-wiki-gigaword-100` | components=10 | 68.43
`uSIF` | `glove-wiki-gigaword-300` | length=11 | 67